In [1]:
import folium
import pandas as pd
import re

In [2]:
# support de réprésentaion géographique des données 
# centrée autour Bordeaux de coordonnées gps : [44.837789, -0.57918]
m = folium.Map(location=[44.837789, -0.57918])
m

In [3]:
# recupération des données scrappé le 8 octobre 2021 sur superimmo.com
df = pd.read_csv('../datas/superimmo/20211008/superimmo_20211008_33.csv')

# repartition des annonces par ville (code postal en fait)
df.location.value_counts()

Bordeaux (33000)                     1367
Mérignac (33700)                      571
Libourne (33500)                      501
Bordeaux (33800)                      487
Bordeaux (33100)                      408
                                     ... 
Saint-Girons-d'Aiguevives (33920)       1
Daubèze (33540)                         1
SAINT GERMAIN D ESTEUIL (33340)         1
Castets et Castillon (33210)            1
Bellefond (33760)                       1
Name: location, Length: 479, dtype: int64

In [4]:
#extraction du code postaux des "location"(exemple:Marseille 8ème (13008)) via une regex
def getcp(location):
    result = re.search(r".*\((.*)\)", location)
    if result:
        return result.group(1)
    

df['Code_postal'] = df.location.apply(lambda x: getcp(str(x)))
# df.info()
# ça n'a pas marché partout
df[(df['Code_postal'].isna())][['location','Code_postal','titre']]
# parce que location n'y ait pas partout ...

# on va les retirer pour le moment mais il ne faut pas qu'on les oublie ...
df = df[-(df['Code_postal'].isna())]

# df.info()

In [5]:
# obtenir les coordonnées gps des codes postaux obtenus
# à partir du fichier pris sur : 
# https://www.data.gouv.fr/fr/datasets/base-officielle-des-codes-postaux/
coo=pd.read_csv('../datas/locations/laposte_hexasmal.csv',sep=";")
#coo.info()

# formatage
coo.Code_postal = coo.Code_postal.astype('str')

### ici je voulais faire une jointure entre les deux fichiers 
### un truc dans le genre : 
### df['gps'] = pd.merge(df, coo, how="left", on=["Code_postal"])
### mais ça ne marchait pas bien 
### 
### J'ai finalement constuit un dictionnaire clé->valeur entre les codes postaux et le coo gps
### il y surement moyen de faire mieux 

cpcoo={}
for cp,coo in zip (coo['Code_postal'].to_list(),coo['coordonnees_gps'].to_list()) :
    cpcoo[cp]=coo

def getcoo(x,cpcoo):       
    try :
        return cpcoo[x];
    except :
         return'0,0';
    

df['gps']=df.Code_postal.apply(lambda x: getcoo(str(x),cpcoo))
print(df[['Code_postal','gps']])


      Code_postal                             gps
0           33830   44.4861918644,-0.786911494042
1           33830   44.4861918644,-0.786911494042
2           33450    44.922024323,-0.355652161244
3           33450    44.922024323,-0.355652161244
4           33830   44.4861918644,-0.786911494042
...           ...                             ...
16562       33000    44.8572445351,-0.57369678116
16563       33230  45.0103439546,-0.0695991585994
16564       33200    44.8572445351,-0.57369678116
16565       33750    44.843131883,-0.284955496145
16566       33490   44.6132843909,-0.232960926916

[16387 rows x 2 columns]


In [6]:
# calcul du prix au metre carrée par code postal
df = df[(df.type=='appartement')]

# seulement si on a biens des valeurs 
df = df[-(df.prix.isna())]
df = df[-(df.surface.isna())]

# formatage
# 120 500 € => 120000 par exemple
df.prix = df.prix.apply(lambda x: int(float(str(x)
                            .replace('€','')
                            .replace(' ','')
                            .replace(',','.') # certaines annonces ont des prix à virgules
                        )))
df.prix = df.prix.astype(int)

# 85,65 => 86.65 par exemple
df.surface = df.surface.apply(lambda x: str(x).replace(' ','').replace(',','.'))
df.surface = df.surface.astype(float)

# deduction du prix au metre carré
df['pm2']=df['prix']/df['surface']

print(df.groupby('Code_postal').describe()['pm2'])



             count          mean          std           min           25%  \
Code_postal                                                                 
33             5.0   5667.528771  1334.599915   3694.331984   4897.959184   
33000        768.0   5413.529640  1430.190621   1805.555556   4471.703980   
33100         78.0   5317.146197  4398.475550   2500.000000   4156.428571   
33110         87.0   4562.370860  1586.856220   2696.629213   3385.555360   
33112          3.0   2111.900070   124.346066   1975.000000   2058.928571   
...            ...           ...          ...           ...           ...   
33920          3.0   1911.020734   212.012643   1667.272727   1840.215311   
33950         10.0   8489.854186  3468.843193   3573.529412   6150.439342   
33970          3.0  12359.642475   807.837874  11428.571429  12102.216749   
33980         16.0   3781.080994   542.075825   2779.411765   3300.000000   
33990          1.0   2498.023256          NaN   2498.023256   2498.023256   

In [7]:
# recensement des données à placer sur la carte regroupées par code postal
dfByCp = df.groupby('Code_postal').agg({'pm2' : ['mean'],     # prix au mettre moyen des annonces
                                        'gps':['min'],        # coordonnées gps (qui sont normalement toutes les mêmes)
                                        'location':['min']    # nom de lieu correspondant au code postal
                                       })
print(dfByCp)

print(dfByCp['pm2']['mean'].min())

                      pm2                            gps  \
                     mean                            min   
Code_postal                                                
33            5667.528771                            0,0   
33000         5413.529640   44.8572445351,-0.57369678116   
33100         5317.146197   44.8572445351,-0.57369678116   
33110         4562.370860  44.8661298323,-0.602121558626   
33112         2111.900070  45.1404040788,-0.862340576235   
...                   ...                            ...   
33920         1911.020734   45.185092781,-0.501188471075   
33950         8489.854186   44.7609440223,-1.19293697437   
33970        12359.642475   44.7609440223,-1.19293697437   
33980         3781.080994  44.7138966628,-0.944173294479   
33990         2498.023256   45.1807913533,-1.06580080764   

                                         location  
                                              min  
Code_postal                                        
33 

In [8]:
# utilisation d'un dégradé de couleur 
# pour indentifier visuellement les prix au mettre carré
# obtenue grace au site : https://colordesigner.io/gradient-generator

gradient40 = ['#fafa6e',
'#faf568',
'#f9f063',
'#f9eb5d',
'#f8e658',
'#f8e153',
'#f7dc4d',
'#f7d748',
'#f6d243',
'#f6cd3f',
'#f5c83a',
'#f4c335',
'#f4bd30',
'#f3b82c',
'#f2b327',
'#f1ae23',
'#f0a91f',
'#efa41b',
'#ee9e16',
'#ed9912',
'#ec940e',
'#eb8e0a',
'#ea8906',
'#e88303',
'#e77e01',
'#e67800',
'#e47300',
'#e36d00',
'#e16700',
'#df6100',
'#de5b00',
'#dc5501',
'#da4f02',
'#d84804',
'#d64105',
'#d43a07',
'#d23209',
'#d0290b',
'#cd1e0d',
'#cb0f0f']

# colorimétrie faite en fractionnant les valeurs 
# sur une échelle de 40 valeurs partagées 
# entre le prix au mettre carré minimum et maximum
mrg=1 # marge pour éviter les sorties mémoires "out of index"
print("repartion géographique des prix au mettre carré")
print("des appartements des annonces de superimmo.com")
print("de la gironde fait le 8 octobre 2021")
print("min : "+str(dfByCp['pm2']['mean'].min()))
print("max : "+str(dfByCp['pm2']['mean'].max()))

for gps,location,pm2 in zip (dfByCp['gps']['min'].to_list(),
                             dfByCp['location']['min'].to_list(),
                             dfByCp['pm2']['mean'].to_list()) :
    folium.CircleMarker(
    location=gps.split(','),
    radius=10,
    popup=location+" : "+str(pm2),
    color=gradient40[int((pm2-dfByCp['pm2']['mean'].min()-mrg)/(dfByCp['pm2']['mean'].max()+100-dfByCp['pm2']['mean'].min()-mrg)*40)], 
    fill=True,
    fill_color=gradient40[int((pm2-dfByCp['pm2']['mean'].min()-mrg)/(dfByCp['pm2']['mean'].max()+100-dfByCp['pm2']['mean'].min()-mrg)*40)],
    ).add_to(m)    
m 

repartion géographique des prix au mettre carré
des appartements des annonces de superimmo.com
de la gironde fait le 8 octobre 2021
min : 581.71875
max : 12359.642474887485
